Código para realizar análisis de sentimientos sobre los comentarios

In [1]:
import pandas as pd
from transformers import pipeline

# Crear un pipeline de análisis de sentimiento
pipe = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

C:\Users\samuel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Mapea los valores de sentimientos a valores numericos
mapeo_valores = {'1 star': 1, '2 stars': 2, '3 stars': 3, '4 stars': 4, '5 stars': 5}

csv_path = 'comments/raquelcanas_comments.csv'

perfil = 'raquelcanas'
data = pd.read_csv(csv_path)

data.head()


,post_id,comment_id,comment_user,comment_text
0,3223656191053441376,17910836606768564,pierettijoyas,✨💫✨💫
1,3223656191053441376,18018130864858089,anabeliiob,Se ve que se divirtieron 🙌❤️
2,3223656191053441376,18032779513620640,kzetinoo,Que lindos❤️
3,3223656191053441376,17988126053230630,1brenda_amaya,Se ve súper ✨✨😀
4,3223656191053441376,18017148979739243,lore.porti,Que lindo Diego de Harry Potter 😊🎃🧡


In [4]:
# Obtener la lista de comentarios y comment_ids
df_list = data['comment_text'].tolist()
comment_ids = data['comment_id'].tolist()

max_comment_length = 512
for comment_id, comment in zip(comment_ids, df_list):
        if isinstance(comment, str) and comment.strip():  # Verificar si el comentario no está vacío
            comment = comment[:max_comment_length] #truncar el comentario si es mas largo que la longitud maxima
            result = pipe(comment)

            print(f"Comment ID: {comment_id}")
            print(f"Comentario: {comment}")
            print(f"Polaridad: {result[0]['label']}, Score: {round(result[0]['score'], 4)}")
            print()

            # Agregar los resultados al DataFrame
            data.loc[data['comment_id'] == comment_id, 'sentiment_label'] = mapeo_valores[result[0]['label']]

        elif isinstance(comment, str) and not comment.strip():
            print(f"Post ID: {comment_id}")
            print("Comentario vacío. No se realiza el análisis de sentimiento.")
            print()
        else:
            print(f"Post ID: {comment_id}")
            print("Comentario no es una cadena de texto. No se realiza el análisis de sentimiento.")
            print()


Comment ID: 17910836606768564
Comentario: ✨💫✨💫
Polaridad: 5 stars, Score: 0.3086

Comment ID: 18018130864858089
Comentario: Se ve que se divirtieron 🙌❤️
Polaridad: 3 stars, Score: 0.3547

Comment ID: 18032779513620640
Comentario: Que lindos❤️
Polaridad: 5 stars, Score: 0.2433

Comment ID: 17988126053230630
Comentario: Se ve súper ✨✨😀
Polaridad: 5 stars, Score: 0.7341

Comment ID: 18017148979739243
Comentario: Que lindo Diego de Harry Potter 😊🎃🧡
Polaridad: 5 stars, Score: 0.4789

Comment ID: 18013169563756158
Comentario: ❤️❤️🔥🔥🔥
Polaridad: 5 stars, Score: 0.3086

Comment ID: 18024271207665779
Comentario: 🙌
Polaridad: 5 stars, Score: 0.3086

Comment ID: 18239308069234174
Comentario: Amo tus botas 😍😍😍😍
Polaridad: 5 stars, Score: 0.4388

Comment ID: 18205743148260201
Comentario: Me encanta este look con botas😍😍😍 se le ve muy comoda ❤️❤️❤️
Polaridad: 5 stars, Score: 0.7841

Comment ID: 18076578784406166
Comentario: Quedaron espectaculares 🔥
Polaridad: 5 stars, Score: 0.6901

Comment ID: 179

KeyboardInterrupt: 

In [ ]:
#  Calcular la media de los resultados por post_id
csv_sentiments = data.groupby('post_id')['sentiment_label'].mean().reset_index()

# Exportar la media de los resultados a un archivo csv
csv_sentiments.to_csv(f'sentiments/{perfil}_sentiments.csv', index=False)

csv_sentiments


,post_id,sentiment_label
0,706127404787215576,4.397959
1,726850003791949657,4.806452
2,750279570149781113,4.490196
3,753968933316504646,4.938776
4,759494888059481892,4.418605
...,...,...
579,3193405042692503921,4.472441
580,3194185283421764050,4.371429
581,3208446869071626006,3.457143
582,3217161309438743288,4.428571
